In [ ]:
# Run the following commands to install InSTA and prepare the environment

# !docker pull brandontrabucco/insta-browser-environment
# !docker run -p 7860:7860 -p 3000-3007:3000-3007 -t brandontrabucco/insta-browser-environment &
# !pip install git+https://github.com/data-for-agents/insta

In [1]:
from insta import (
    InstaPipeline,
    create_demo_videos
)

In [2]:
pipeline = InstaPipeline()

In [3]:
for data_idx, trajectory in enumerate(
    pipeline.iter_pipeline()
):

    if data_idx >= 10:
        
        break

Processing: dutchartinstitute.eu:   0%|                                                                  | 6/143811 [20:34<8219:38:58, 205.77s/it]


KeyboardInterrupt: 

In [4]:
create_demo_videos()

Processing: research-portal.uws.ac.uk: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:12<00:00,  2.10s/it]


In [7]:
from ipywidgets import Output, GridspecLayout
from IPython import display

import glob
import json
import os

selected_video_files = glob.glob(
    "data/videos/*.mp4"
)

first_video_path = selected_video_files[0]

# show the final observation, action, and judgment responses:

final_observation = "data/observations/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(final_observation, "r") as file:

    final_observation = json.load(file)[-1]

final_action = "data/actions/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(final_action, "r") as file:

    final_action = json.load(file)[-1]

judgment = "data/judgments/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(judgment, "r") as file:

    judgment = json.load(file)

print(
    "Final Observation for {}:\n\n{}\n\n".format(
        first_video_path,
        final_observation["processed_text"]
    )
)

print(
    "Final Action for {}:\n\n{}\n\n".format(
        first_video_path,
        final_action["response"]
    )
)

print(
    "Judgment for {}:\n\n{}\n\n".format(
        first_video_path,
        judgment["response"]
    )
)

# display videos:

height = len(selected_video_files)

grid = GridspecLayout(
    1, height
)

for panel_idx, video_path in enumerate(
    selected_video_files
):
    
    video_output = Output()
    
    with video_output:
        
        display.display(display.Video(
            video_path,
            embed = True,
            width = 1000
        ))
        
    grid[0, panel_idx] = video_output

grid

Final Observation for data/videos/touchstoneenergy.com.mp4:

Efficiency | Touchstone Energy Cooperative, Inc 101 Easy Ways To Save - Fridge Seal 101 Easy Ways To Save - Seal Gaps 101 Easy Ways To Save - Water Heater 101 Ways To Save - Blinds 101 Ways To Save - Thermostat 
## Reliable Information About Energy Choices
 
* [id: 930] Quick Tips link
* [id: 932] Home Energy Adventure link
* [id: 934] DIY link
 
### Do It Yourself
 Explore more detailed projects for upgrading the energy efficiency of your home. Whether a DIY’er or working with a contractor, these documents help provide detailed guidance. 
1. [id: 1026] Attic link
2. [id: 1050] Basement link
3. [id: 1074] Energy Efficiency link
4. [id: 1081] Do it Yourself Home Energy Audit link ( [id: 1082] English link ) ( [id: 1083] Spanish link )
5. [id: 1085] Indoor Air Quality link ( [id: 1086] English link ) ( [id: 1087] Spanish link )
6. [id: 1089] Lighting Your House link ( [id: 1090] English link ) ( [id: 1091] Spanish link )
7. [id

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…